# Setup

In [ ]:
videos_dir = None
save_dir = None

In [ ]:
import os

dir_path = os.getcwd()

if not videos_dir:
    if 'google.colab' in str(get_ipython()):
        # Update this path as necessary
        videos_dir = f'{dir_path}/AIC_Video'
    elif 'kaggle' in str(get_ipython()):
        videos_dir = f'{dir_path}/AIC_Video'
    else:
        parent_dir_path = os.path.dirname(dir_path)
        videos_dir = f'{parent_dir_path}/dataset/AIC_Video'
        
if not save_dir:
    save_dir = f'{dir_path}/SceneJSON'

In [ ]:
!git clone https://github.com/soCzech/TransNetV2.git
!git lfs install
!cd TransNetV2
!git lfs fetch https://github.com/soCzech/TransNetV2.git
!git lfs checkout

fatal: destination path 'TransNetV2' already exists and is not an empty directory.


Updated Git hooks.
Git LFS initialized.
fetch: Fetching reference refs/heads/feat/metadata


In [ ]:
# Import module
import os
import cv2
import json
import glob
import ffmpeg
import torch
import numpy as np
from tqdm import tqdm
from TransNetV2.inference.transnetv2 import TransNetV2

2024-07-28 12:38:03.625608: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-28 12:38:03.668801: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-28 12:38:04.440058: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Parse video info

In [ ]:
def parse_video_info(videos_dir):
    """Parse video information from the directory structure."""
    all_video_paths = {}
    for part in sorted(os.listdir(videos_dir)):
        data_part = part.split('_')[-1]  # L01, L02 for example
        all_video_paths[data_part] = {}

    for data_part in sorted(all_video_paths.keys()):
        data_part_path = f'{videos_dir}/Videos_{data_part}/video'
        video_paths = sorted(os.listdir(data_part_path))
        video_ids = [video_path.replace('.mp4', '').split('_')[-1] for video_path in video_paths]
        for video_id, video_path in zip(video_ids, video_paths):
            video_path_full = f'{data_part_path}/{video_path}'
            all_video_paths[data_part][video_id] = video_path_full

    return all_video_paths

# Inference

In [ ]:
def create_directory(path):
    """Create a directory if it does not exist."""
    if not os.path.exists(path):
        os.makedirs(path)

def predict_scenes_and_save(model, all_video_paths, save_dir):
    """Predict scenes for each video and save the results as JSON files."""
    create_directory(save_dir)
    
    cnt = 0

    for key, video_paths_dict in tqdm(all_video_paths.items()):      
        video_ids = sorted(video_paths_dict.keys())
        key_dir = os.path.join(save_dir, key)
        create_directory(key_dir)

        for video_id in tqdm(video_ids, desc=f"Processing {key}"):            
            if cnt == 5:
                break
            cnt += 1
                
            video_path = video_paths_dict[video_id]
            _, single_frame_predictions, _ = model.predict_video(video_path)

            # Generate list of scenes from predictions, returns tuples of (start frame, end frame)
            scenes = model.predictions_to_scenes(single_frame_predictions)

            with open(os.path.join(key_dir, f"{video_id}.json"), 'w') as f:
                json.dump(scenes.tolist(), f)

In [ ]:
model = TransNetV2()
all_video_paths = parse_video_info(videos_dir)
predict_scenes_and_save(model, all_video_paths, save_dir)

[TransNetV2] Using weights from /home/jiggle/personal/competition/hcm-ai/Pipeline_HCM_AI/notebooks/data_extraction/transnet/TransNetV2/inference/transnetv2-weights/.


2024-07-28 12:38:06.053238: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 12:38:06.063594: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
  0%|          | 0/2 [00:00<?, ?it/s]

[TransNetV2] Extracting frames from /home/jiggle/personal/competition/hcm-ai/Pipeline_HCM_AI/notebooks/data_extraction/transnet/AIC_Video/Videos_L01/video/L01_V001.mp4


2024-07-28 12:38:37.206564: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [1,100,27,48,3]
	 [[{{node inputs}}]]
2024-07-28 12:38:37.786374: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 33177600 exceeds 10% of free system memory.
2024-07-28 12:38:37.923732: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 33177600 exceeds 10% of free system memory.


[TransNetV2] Processing video frames 50/27714

...

[TransNetV2] Processing video frames 22867/22867
[TransNetV2] Extracting frames from /home/jiggle/personal/competition/hcm-ai/Pipeline_HCM_AI/notebooks/data_extraction/transnet/AIC_Video/Videos_L21/video/L21_V004.mp4
[TransNetV2] Processing video frames 26000/26045

[TransNetV2] Processing video frames 26045/26045
[TransNetV2] Extracting frames from /home/jiggle/personal/competition/hcm-ai/Pipeline_HCM_AI/notebooks/data_extraction/transnet/AIC_Video/Videos_L21/video/L21_V005.mp4
[TransNetV2] Processing video frames 19300/19314

[TransNetV2] Processing video frames 19314/19314
[TransNetV2] Extracting frames from /home/jiggle/personal/competition/hcm-ai/Pipeline_HCM_AI/notebooks/data_extraction/transnet/AIC_Video/Videos_L21/video/L21_V006.mp4


 50%|█████     | 1/2 [29:38<29:38, 1778.88s/it]


KeyboardInterrupt: 